# Sources ([scarlet.sources](source.ipynb))

## Getting Started

The base [Source](#scarlet.source.Source) class requires an initial guess at the SED and morphology of the sources (or a list of seds and morphologies if the source has mutliple components, eg. a bulge and a disk).
For most general blends neither the SED nor the morphology are known ahead of time so a set of convenience classes: [PointSource](source.ipynb#scarlet.source.PointSource) and [ExtendedSource](source.ipynb#scarlet.source.ExtendedSource) exist to generate an initial morpholgy and SED for a source at a given location in the image.
However, it is also possible for users to specify their own source types with custom initialization methods.

See [Sources](overview.ipynb#Sources) for more on initializing and using sources.